In [0]:
pip install sgp4


In [0]:
dbutils.library.restartPython()

In [0]:
# sample
from sgp4.api import Satrec
from sgp4.api import jday

# Paste your TLE here (2 lines)
line1 = "1 25544U 98067A   18264.51782528  .00002182  00000-0  11606-4 0  2927"
line2 = "2 25544  51.6416 247.4627 0006703 130.5360 325.0288 15.72125391563537"

# Create satellite record from TLE lines
satellite = Satrec.twoline2rv(line1, line2)

# Extract epoch from the satellite record (year, month, day, hour, minute, second)
year, month, day, hour, minute, second = satellite.jdsatepoch, satellite.jdsatepochF, 0, 0, 0, 0

# satellite.jdsatepoch is Julian date integer part, jdsatepochF is fractional part
jd = satellite.jdsatepoch
jd_frac = satellite.jdsatepochF

# Convert Julian date to calendar date (using sgp4's jday function in reverse)
# We approximate here; better to use an external julian date converter, or:
# We'll use jday for the epoch date itself:
# But sgp4 does not provide inverse, so let's use an external package:
from sgp4.api import Satrec, WGS72
from datetime import datetime, timedelta

def jd_to_datetime(jd):
    # Julian day to datetime conversion
    # JD 2451545.0 = 2000-01-01 12:00:00 UTC
    JD_UNIX_EPOCH = 2440587.5
    unix_time = (jd - JD_UNIX_EPOCH) * 86400.0
    return datetime.utcfromtimestamp(unix_time)

epoch_datetime = jd_to_datetime(jd + jd_frac)

print(f"Epoch datetime (UTC): {epoch_datetime}")

# Compute position and velocity at the epoch time (which is the TLE epoch)
e, r, v = satellite.sgp4(jd, jd_frac)

if e == 0:
    # r is position vector [km], v is velocity vector [km/s]
    print(f"Position (ECI) in km: x={r[0]:.3f}, y={r[1]:.3f}, z={r[2]:.3f}")
    print(f"Velocity (ECI) in km/s: vx={v[0]:.6f}, vy={v[1]:.6f}, vz={v[2]:.6f}")
else:
    print(f"Error in propagation: error code {e}")


In [0]:
%python
from sgp4.api import Satrec
from sgp4.api import jday
from datetime import datetime, timedelta

# Read the table into a Spark DataFrame
df = spark.table("workspace.default.gp_latest_20250810_221255s")

# Select the first row for demonstration purposes
tle_data = df.select("TLE_LINE1", "TLE_LINE2").first()

# Extract TLE lines
line1 = tle_data["TLE_LINE1"]
line2 = tle_data["TLE_LINE2"]

# Create satellite record from TLE lines
satellite = Satrec.twoline2rv(line1, line2)

# Extract epoch from the satellite record (year, month, day, hour, minute, second)
year, month, day, hour, minute, second = satellite.jdsatepoch, satellite.jdsatepochF, 0, 0, 0, 0

# satellite.jdsatepoch is Julian date integer part, jdsatepochF is fractional part
jd = satellite.jdsatepoch
jd_frac = satellite.jdsatepochF

# Convert Julian date to calendar date
def jd_to_datetime(jd):
    # Julian day to datetime conversion
    # JD 2451545.0 = 2000-01-01 12:00:00 UTC
    JD_UNIX_EPOCH = 2440587.5
    unix_time = (jd - JD_UNIX_EPOCH) * 86400.0
    return datetime.utcfromtimestamp(unix_time)

epoch_datetime = jd_to_datetime(jd + jd_frac)

print(f"Epoch datetime (UTC): {epoch_datetime}")

# Compute position and velocity at the epoch time (which is the TLE epoch)
e, r, v = satellite.sgp4(jd, jd_frac)

if e == 0:
    # r is position vector [km], v is velocity vector [km/s]
    print(f"Position (ECI) in km: x={r[0]:.3f}, y={r[1]:.3f}, z={r[2]:.3f}")
    print(f"Velocity (ECI) in km/s: vx={v[0]:.6f}, vy={v[1]:.6f}, vz={v[2]:.6f}")
else:
    print(f"Error in propagation: error code {e}")

In [0]:
%python
from sgp4.api import Satrec
from datetime import datetime, timedelta
import pandas as pd

# Read the table into a Spark DataFrame
df = spark.table("workspace.default.gp_latest_20250810_221255s")

# Convert Spark DataFrame to Pandas DataFrame for easier manipulation
df_pd = df.toPandas()

# Function to convert Julian date to datetime
def jd_to_datetime(jd, jd_frac):
    JD_UNIX_EPOCH = 2440587.5
    unix_time = (jd + jd_frac - JD_UNIX_EPOCH) * 86400.0
    return datetime.utcfromtimestamp(unix_time)

# Function to compute position and velocity from TLE lines
def compute_position_velocity(row):
    satellite = Satrec.twoline2rv(row['TLE_LINE1'], row['TLE_LINE2'])
    jd = satellite.jdsatepoch
    jd_frac = satellite.jdsatepochF
    epoch_datetime = jd_to_datetime(jd, jd_frac)
    e, r, v = satellite.sgp4(jd, jd_frac)
    if e == 0:
        return pd.Series({
            'epoch_datetime': epoch_datetime,
            'pos_x_km': r[0], 'pos_y_km': r[1], 'pos_z_km': r[2],
            'vel_x_kms': v[0], 'vel_y_kms': v[1], 'vel_z_kms': v[2]
        })
    else:
        return pd.Series({
            'epoch_datetime': epoch_datetime,
            'pos_x_km': None, 'pos_y_km': None, 'pos_z_km': None,
            'vel_x_kms': None, 'vel_y_kms': None, 'vel_z_kms': None
        })

# Apply the function to each row in the DataFrame
parsed_df = df_pd.apply(compute_position_velocity, axis=1)

# Join the parsed data with the original DataFrame
df_pd = df_pd.join(parsed_df)

# Convert back to Spark DataFrame
df_spark = spark.createDataFrame(df_pd)

# # Save as Delta table
# yesterday = datetime.utcnow() - timedelta(days=1)
table_name = f"gp_latest_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}s"
df_spark.write.format("delta").mode("overwrite").saveAsTable(table_name)

# Display the DataFrame
display(df_spark)

In [0]:
%python
import matplotlib.pyplot as plt

# Plot position (ECI) for all satellites
plt.figure(figsize=(10, 6))
plt.scatter(df_pd['pos_x_km'], df_pd['pos_y_km'], c='blue', alpha=0.5)
plt.xlabel('ECI X Position (km)')
plt.ylabel('ECI Y Position (km)')
plt.title('Satellite ECI Position (X vs Y)')
plt.grid(True)
plt.show()

In [0]:
%python
import matplotlib.pyplot as plt
import numpy as np

# Function to plot position and velocity
def plot_position_velocity(df):
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Plot Earth as a sphere (radius ~6371 km)
    earth_radius = 6371
    u = np.linspace(0, 2 * np.pi, 100)
    v_sphere = np.linspace(0, np.pi, 100)
    x = earth_radius * np.outer(np.cos(u), np.sin(v_sphere))
    y = earth_radius * np.outer(np.sin(u), np.sin(v_sphere))
    z = earth_radius * np.outer(np.ones(np.size(u)), np.cos(v_sphere))

    ax.plot_surface(x, y, z, color='b', alpha=0.3)

    # Plot satellite positions
    ax.scatter(df['pos_x_km'], df['pos_y_km'], df['pos_z_km'], color='r', s=10, label='Satellite Position')

    # Plot velocity vectors
    for i in range(len(df)):
        ax.quiver(df['pos_x_km'][i], df['pos_y_km'][i], df['pos_z_km'][i],
                  df['vel_x_kms'][i], df['vel_y_kms'][i], df['vel_z_kms'][i],
                  color='g', length=1000, normalize=True, label='Velocity Vector' if i == 0 else "")

    # Set labels and limits
    ax.set_xlabel('X (km)')
    ax.set_ylabel('Y (km)')
    ax.set_zlabel('Z (km)')
    ax.set_title('Satellite Position and Velocity in ECI Frame\n(Earth at Origin)')

    max_val = earth_radius + 1000  # slightly larger than earth radius
    lims = max(abs(df[['pos_x_km', 'pos_y_km', 'pos_z_km']]).max().max(), max_val)
    ax.set_xlim([-lims, lims])
    ax.set_ylim([-lims, lims])
    ax.set_zlim([-lims, lims])

    ax.legend()
    plt.show()

# Convert Spark DataFrame to Pandas DataFrame for plotting
df_pd = df_spark.toPandas()

# Plot the position and velocity
plot_position_velocity(df_pd)

In [0]:
# Plotting sample with respect to earth;s position
from sgp4.api import Satrec
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

# TLE lines
line1 = "1 25544U 98067A   18264.51782528  .00002182  00000-0  11606-4 0  2927"
line2 = "2 25544  51.6416 247.4627 0006703 130.5360 325.0288 15.72125391563537"

# Create satellite record
satellite = Satrec.twoline2rv(line1, line2)

jd = satellite.jdsatepoch
jd_frac = satellite.jdsatepochF

# Propagate satellite position and velocity at epoch
e, r, v = satellite.sgp4(jd, jd_frac)

if e != 0:
    raise RuntimeError(f"Error in SGP4 propagation: {e}")

# Convert Julian date to datetime for printing
def jd_to_datetime(jd):
    JD_UNIX_EPOCH = 2440587.5
    unix_time = (jd - JD_UNIX_EPOCH) * 86400.0
    return datetime.utcfromtimestamp(unix_time)

epoch_datetime = jd_to_datetime(jd + jd_frac)
print(f"Epoch datetime (UTC): {epoch_datetime}")

# Position vector r in km, velocity vector v in km/s
r = np.array(r)
v = np.array(v)

print(f"Position (ECI) [km]: {r}")
print(f"Velocity (ECI) [km/s]: {v}")

# --- Plotting ---
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')

# Plot Earth as a sphere (radius ~6371 km)
earth_radius = 6371
u = np.linspace(0, 2 * np.pi, 100)
v_sphere = np.linspace(0, np.pi, 100)
x = earth_radius * np.outer(np.cos(u), np.sin(v_sphere))
y = earth_radius * np.outer(np.sin(u), np.sin(v_sphere))
z = earth_radius * np.outer(np.ones(np.size(u)), np.cos(v_sphere))

ax.plot_surface(x, y, z, color='b', alpha=0.3)

# Plot satellite position
ax.scatter(r[0], r[1], r[2], color='r', s=100, label='Satellite Position')

# Plot velocity vector starting at satellite position (scale velocity for visualization)
velocity_scale = 2000  # scale km/s to km for visibility
v_scaled = v * velocity_scale

ax.quiver(r[0], r[1], r[2],
          v_scaled[0], v_scaled[1], v_scaled[2],
          color='g', length=np.linalg.norm(v_scaled), normalize=True, label='Velocity Vector')

# Set labels and limits
ax.set_xlabel('X (km)')
ax.set_ylabel('Y (km)')
ax.set_zlabel('Z (km)')
ax.set_title('Satellite Position and Velocity in ECI Frame\n(Earth at Origin)')

max_val = earth_radius + 1000  # slightly larger than earth radius
lims = max(abs(r).max(), max_val)
ax.set_xlim([-lims, lims])
ax.set_ylim([-lims, lims])
ax.set_zlim([-lims, lims])

ax.legend()
plt.show()
